In [2]:
from nlu_engine import EntityExtractor, crf
from nlu_engine import Analytics
from nlu_engine import RenderJSON
from nlu_engine import DataUtils
from nlu_engine import MacroEntityRefinement
from nlu_engine import MacroDataRefinement

from nlu_engine import NLUEngine
import nltk
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
#TODO: redo intro to be specific for general reports

# Macro NLU Entity Refinement

It's a bit like the TV show [Serverance](https://www.imdb.com/title/tt11280740/) .

![Helly R and Mark S](https://media.npr.org/assets/img/2022/02/15/atv_severance_photo_010103-5f8033cc2b219ba64fe265ce893eae4c90e83896-s1100-c50.jpg "Helly R and Mark G")

*Helly R*: `My job is to scroll through the spreadsheet and look for the numbers that feel scary?`

*Mark S*: `I told you, you’ll understand when you see it, so just be patient.`

![MDR](https://www.imore.com/sites/imore.com/files/styles/large/public/field/image/2022/03/refinement-software-severance-apple-tv.jpg "serverance micro data refinement")

*Helly R*: `That was scary. The numbers were scary.`

Hopefully the intents and entities that are wrong aren't scary, just a bit frustrating. Let's see if we can find the right ones.

NOTE: We will use Logistic Regression with TFIDF features to train our intent models and CRFs for entity exraction. Why? Well, they are very fast and both methods aren't state-of-the-art. This is good, because it is easier to find problems we will need to refine in the dataset than if we were to use a proper NLU engine like Snips or something SOTA like BERT. It is very important to note that some of the the problems we will pick up on, might not be an actual issue, but might be due to the limitations of the models. Refining the real problems and ignoring the limitations of the models is a good way to improve the models. Then when the dataset is ready, we can use some more advanced NLU engine and get the best performance possible.

* Macro NLU Data Refinement: Intent
* Macro NLU Data Refinement: Entity


Load the dataset

In [ ]:
#TODO: figure out what needs to be done here.

In [3]:
try:
    nlu_data_df = pd.read_csv(
        'data/refined/nlu_data_refined_df.csv', sep=',', index_col=0)
    print('Successfully loaded nlu_data_refined_df.csv')
except:
    data = 'data/NLU-Data-Home-Domain-Annotated-All-Cleaned.csv'
    nlu_data_df = DataUtils.load_data(
    data
)

Successfully loaded nlu_data_refined_df.csv


Make sure `nlu_data_df['answer_normalised']` is taken from `nlu_data_df['answer_annotation']`

In [4]:
nlu_data_df = DataUtils.convert_annotated_utterances_to_normalised_utterances(
    nlu_data_df)


We should remove the unwanted entries for the next few steps.

In [5]:
removed_nlu_data_refined_df = nlu_data_df[nlu_data_df['remove'] != True]

# Entity extraction report

The entity extraction could be greatly improved by improving the features it uses. It would be great if someone would take a look at this. Perhaps the CRF features similar to what Snips uses would be better such as Brown clustering (probably).

It is important to have the NLTK tokenizer to be able to extract entities.

In [ ]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
        nltk.download('punkt')
        nltk.download('averaged_perceptron_tagger')

We can create a new entity report or load a previous one.

Load a previous report

In [5]:
entity_report_df = pd.read_csv('data/reports/entity_report.csv')

Create a new entity report

Due to this error featured in [this git issue](https://github.com/TeamHG-Memex/sklearn-crfsuite/issues/60) we have to use an older version of scikit learn (sklearn<0.24), otherwise the latest version would work. Hopefully this gets fixed one day..

In [ ]:
entity_report_df = NLUEngine.evaluate_entity_classifier(
    data_df=removed_nlu_data_refined_df, cv=4)

In [ ]:
entity_report_df

We can save the current report if we wish.

In [ ]:
entity_report_df.to_csv(
    'data/reports/entity_report.csv',
    index=False)

We will need to extract out the entity types from the dataset. We need this to calculate the precentage of entities, to filter out any domains that have too few entities for benchmarking by domain instead of all domains together.

In [7]:
nlu_data_df['entities'] = nlu_data_df['answer_annotation'].apply(
    EntityExtractor.extract_entities)

nlu_data_df['entity_types'] = nlu_data_df['entities'].apply(
    EntityExtractor.extract_entity_types)

import numpy as np
nlu_data_df['entity_types'] = nlu_data_df['entity_types'].apply(lambda y: np.nan if len(y)==0 else y)
nlu_data_df['entities'] = nlu_data_df['entities'].apply(lambda y: np.nan if len(y)==0 else y)

removed_nlu_data_df = nlu_data_df[nlu_data_df['remove'] != True]

We will have a look at the domain entity reports. We can load a previous report or create a new one.

Load a previous report

In [8]:
domain_entity_reports_df = pd.read_csv('data/reports/domain_entity_reports.csv')

Create a new report

In [ ]:
domain_entity_reports_df = NLUEngine.get_entity_reports_for_domains(removed_nlu_data_refined_df)

domain_entity_reports_df = domain_entity_reports_df[domain_entity_reports_df['entity-type'].str.contains(
    'weighted avg')].sort_values(by='f1-score', ascending=True)

In [9]:
domain_entity_reports_df

,entity-type,precision,recall,f1-score,support,domain
0,weighted avg,0.644421,0.685911,0.659549,11051.0,play
1,weighted avg,0.684923,0.782016,0.725743,367.0,music
2,weighted avg,0.838414,0.846966,0.839535,2274.0,takeaway
3,weighted avg,0.844779,0.866175,0.852882,3527.0,recommendation
4,weighted avg,0.884246,0.888836,0.885063,18819.0,calendar
5,weighted avg,0.890012,0.892921,0.888886,4492.0,news
6,weighted avg,0.881286,0.900264,0.889026,2647.0,lists
7,weighted avg,0.878675,0.905823,0.890492,1975.0,cooking
8,weighted avg,0.909339,0.915642,0.912069,6508.0,weather
9,weighted avg,0.915545,0.921031,0.916768,4850.0,iot


Save the report

In [ ]:
domain_entity_reports_df.to_csv('data/reports/domain_entity_reports.csv', index=False)

In [ ]:
#TODO: create new notebook for domain specific entity reports and entity refinement
#TODO: Domain specific entity refinement

In [6]:
nlu_data_df

,userid,answerid,notes,question,suggested_entities,answer,answer_normalised,scenario,intent,predicted_label,intent_refined,entity_refined,remove,status,answer_annotation
0,1.0,1.0,NaN,Write what you would tell your PDA in the foll...,"date, time",wake me up at 5am this week,wake me up at five am this week,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,wake me up at [time : five am] [date : this week]
1,1.0,2.0,NaN,Write what you would tell your PDA in the foll...,"date, time",wake me up at 9am on Friday,wake me up at nine am on friday,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,wake me up at [time : nine am] on [date : friday]
2,1.0,3.0,NaN,Write what you would tell your PDA in the foll...,"date, time",set an alarm for two hours from now,set an alarm for two hours from now,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,set an alarm for [time : two hours from now]
3,1.0,31.0,NaN,Write what you would tell your PDA in the foll...,player_setting,Olly quiet!,quiet,audio,volume_mute,volume_mute,NaN,NaN,NaN,NaN,quiet
5,1.0,33.0,NaN,Write what you would tell your PDA in the foll...,player_setting,Olly pause for ten seconds,pause for ten seconds,audio,volume_mute,volume_mute,True,NaN,True,NaN,pause for [time : ten seconds]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25711,NaN,781.0,NaN,NaN,NaN,"that's cool, musch appreciated, olly.","that's cool, musch appreciated, olly.",general,praise,praise,NaN,NaN,NaN,NaN,"that's cool, musch appreciated, olly."
25712,NaN,782.0,NaN,NaN,NaN,"you are hero, appreciated.","you are hero, appreciated.",general,praise,praise,NaN,NaN,NaN,NaN,"you are hero, appreciated."
25713,NaN,783.0,NaN,NaN,NaN,"thanks, that's nice.","thanks, that's nice.",general,praise,praise,NaN,NaN,NaN,NaN,"thanks, that's nice."
25714,NaN,784.0,NaN,NaN,NaN,"that's cool, thank you so much.","that's cool, thank you so much.",general,praise,praise,NaN,NaN,NaN,NaN,"that's cool, thank you so much."


In [12]:
removed_nlu_data_refined_df = nlu_data_df[nlu_data_df['remove'] != True]


In [ ]:
#TODO: load domain input and get analytics of that domain
#TODO: refine by that domain for each intent (?)

# Domain specific entity analysis

For a new domain start here!

Pick a domain to start.

In [ ]:
domain_selection = MacroDataRefinement.list_and_select_domain(
    removed_nlu_data_refined_df)

domain_df = DataUtils.get_domain_df(
    removed_nlu_data_refined_df, domain_selection)

domain_df

In [ ]:
#TODO: domain specific entity analysis and refinement